# 11. Run Matchups
- Date: 6/13/2024
- Description: This recreates matchup files used as model inputs for yesterday's games

### Imports

In [ ]:
if "imported" not in globals():
    %run "C:\Users\james\Documents\MLB\Code\U1. Imports.ipynb"
    %run "C:\Users\james\Documents\MLB\Code\U2. Utilities.ipynb"
    %run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"

### Games

In [ ]:
start_date = yesterdaysdate
end_date = yesterdaysdate

Read in games

In [ ]:
if "game_df" not in globals():
    game_df = read_and_save_games(team_map, generate=True)
    game_df = game_df[(game_df['date'] >= start_date) & (game_df['date'] <= end_date)].reset_index(drop=True)

### B01. Matchups

In [ ]:
%run "C:\Users\james\Documents\MLB\Code\U4. Datasets.ipynb"
%run "C:\Users\james\Documents\MLB\Code\B01. Matchups.ipynb"

In [ ]:
# Read in park factors
multiplier_df = pd.read_csv(os.path.join(baseball_path, "Multiplier Dataset.csv"))

In [ ]:
%%time
# Read in dataset 
complete_dataset = create_pa_inputs(multiplier_df, 2015, 2024, 50, 300, True)
# Subset
complete_dataset = complete_dataset.query('date > 20210301')

# Read in Steamer hitters
steamer_hitters_df = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_hitters_weekly_log.csv"), encoding='iso-8859-1')
steamer_hitters_df_current = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_hitters.csv"), encoding='iso-8859-1')
steamer_hitters_df = pd.concat([steamer_hitters_df, steamer_hitters_df_current], axis=0)
steamer_hitters_df['proj_year'].fillna(2024, inplace=True)
steamer_hitters_df['proj_date'].fillna(todaysdate_dash, inplace=True)
steamer_hitters_df = clean_steamer_hitters(steamer_hitters_df)

# Read in Steamer pitchers
steamer_pitchers_df = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_pitchers_weekly_log.csv"), encoding='iso-8859-1')
steamer_pitchers_df_current = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_pitchers.csv"), encoding='iso-8859-1')
steamer_pitchers_df = pd.concat([steamer_pitchers_df, steamer_pitchers_df_current], axis=0)
steamer_pitchers_df['proj_year'].fillna(2024, inplace=True)
steamer_pitchers_df['proj_date'].fillna(todaysdate_dash, inplace=True)
steamer_pitchers_df = clean_steamer_pitchers(steamer_pitchers_df)

In [ ]:
# Write to CSV (we'll read these later in B.)
complete_dataset.to_csv(os.path.join(baseball_path, "Complete Dataset.csv"), index=False)
steamer_hitters_df.to_csv(os.path.join(baseball_path, "Steamer Hitters.csv"), index=False)
steamer_pitchers_df.to_csv(os.path.join(baseball_path, "Steamer Pitchers.csv"), index=False)

In [ ]:
# Shrink datasets for faster processing
complete_dataset.drop(columns={'description', 'batterName', 'pitcherName', 'postOnFirst', 'postOnSecond', 'postOnThird', 'preOnFirst', 'preOnSecond', 'preOnThird', 'pitch_name'}, inplace=True)
complete_dataset = complete_dataset.query(f'date > {game_df["date"].astype(int).min()-10000}')
steamer_hitters_df = steamer_hitters_df.query(f'date > {game_df["date"].astype(int).min()-10000}')
steamer_pitchers_df = steamer_pitchers_df.query(f'date > {game_df["date"].astype(int).min()-10000}')

In [ ]:
%%time
print(len(game_df))
empty_list = Parallel(n_jobs=-1, verbose=True)(delayed(create_matchup_files)(game_df, row, complete_dataset, steamer_hitters_df, steamer_pitchers_df, team_map) for row in range(len(game_df)))